In [383]:
import eikon as ek
import numpy as np
import pandas as pd
import json
import datetime as dt
import traceback
import math

# Load & Clean Data

First we merge all the price data together

In [384]:
t1 = pd.read_csv("price_data_c1.csv")
t2 = pd.read_csv("price_data_c2.csv")
t3 = pd.read_csv("price_data_c3.csv")
t4 = pd.read_csv("price_data_c4.csv")

t1.drop(columns=["Unnamed: 0"], inplace=True)
t2.drop(columns=["Unnamed: 0"], inplace=True)
t3.drop(columns=["Unnamed: 0"], inplace=True)
t4.drop(columns=["Unnamed: 0"], inplace=True)

t1["Instrument"] = t1["Instrument"].apply(lambda x: x[:-2])
t2["Instrument"] = t2["Instrument"].apply(lambda x: x[:-2])
t3["Instrument"] = t3["Instrument"].apply(lambda x: x[:-2])
t4["Instrument"] = t4["Instrument"].apply(lambda x: x[:-2])

t2.rename(
    columns={
        "Accumulated Volume": "C2 - Accumulated Volume",
        "Close Price": "C2 - Close Price"
    },
    inplace=True
)

t3.rename(
    columns={
        "Accumulated Volume": "C3 - Accumulated Volume",
        "Close Price": "C3 - Close Price"
    },
    inplace=True
)

t4.rename(
    columns={
        "Accumulated Volume": "C4 - Accumulated Volume",
        "Close Price": "C4 - Close Price"
    },
    inplace=True
)

In [385]:
price_data = t1.merge(t2, how="left").merge(t3, how="left").merge(t4, how="left")
price_data.Date = pd.to_datetime(price_data.Date)
price_data = price_data[~pd.isnull(price_data["C2 - Close Price"])]

In [386]:
len(price_data.Instrument.unique())

27

In [387]:
price_data.Date.min()

Timestamp('1991-09-26 00:00:00')

In [388]:
price_data.Date.max()

Timestamp('2021-12-31 00:00:00')

In [389]:
price_data

,Instrument,Date,Close Price,Accumulated Volume,C2 - Close Price,C2 - Accumulated Volume,C3 - Close Price,C3 - Accumulated Volume,C4 - Close Price,C4 - Accumulated Volume
18,SB,1991-10-01,8.880,15.0,8.770,2007.0,8.730,549.0,8.730,549.0
19,SB,1991-10-02,8.850,152.0,8.740,1317.0,8.720,430.0,8.720,430.0
20,SB,1991-10-03,8.550,82.0,8.480,3825.0,8.480,865.0,8.480,865.0
21,SB,1991-10-04,8.660,52.0,8.590,915.0,8.550,74.0,8.550,74.0
22,SB,1991-10-07,8.750,27.0,8.650,1109.0,8.640,110.0,8.640,110.0
...,...,...,...,...,...,...,...,...,...,...
184232,SI,2021-12-27,22.680,1.0,23.060,108793.0,23.100,111.0,23.100,111.0
184233,SI,2021-12-28,23.150,211.0,23.015,73451.0,23.100,160.0,23.100,160.0
184234,SI,2021-12-29,22.635,172.0,22.825,98976.0,22.860,45.0,22.860,45.0
184235,SI,2021-12-30,23.070,344.0,23.100,75359.0,23.075,109.0,23.075,109.0


In [390]:
price_data[price_data.Instrument == "SB"].to_csv("sugar_no_11_test.csv")

In [391]:
price_data.columns

Index(['Instrument', 'Date', 'Close Price', 'Accumulated Volume',
       'C2 - Close Price', 'C2 - Accumulated Volume', 'C3 - Close Price',
       'C3 - Accumulated Volume', 'C4 - Close Price',
       'C4 - Accumulated Volume'],
      dtype='object')

Next, we perform manual sanity checks to see from which dates we can start our analysis for each futures contract

In [392]:
price_data.to_csv("price_data_check.csv", index=False)

In [393]:
price_data = price_data[
    (price_data.Instrument != "RB") |
    (price_data.Date >= pd.to_datetime("2006-04-03"))
]

In [394]:
price_data = price_data[
    (price_data.Instrument != "SAF") |
    (price_data.Date >= pd.to_datetime("1995-01-01"))
]

In [395]:
price_data = price_data[
    (price_data.Instrument != "SI") |
    (price_data.Date >= pd.to_datetime("1991-11-11"))
]

In [396]:
start_dates = price_data.groupby("Instrument").agg(min_date=("Date","min")).reset_index()

In [397]:
# Each contract starts 1 year after its manually validated dates
start_dates.min_date = start_dates.min_date.apply(lambda x: x.year + 1)

In [398]:
start_dates

,Instrument,min_date
0,C,1992
1,CC,1992
2,CL,1992
3,CT,1992
4,FC,1992
5,GC,1992
6,HG,1992
7,HO,1992
8,ICF,2000
9,KC,1992


In [399]:
start_dates.to_csv("start_dates.csv")

Finally, we have to use these start years to clean up our price dataset

In [400]:
price_data = price_data.merge(start_dates)
price_data = price_data[price_data.Date.dt.year >= price_data.min_date].copy()
price_data.drop(columns=["min_date"], inplace=True)

In [401]:
price_data

,Instrument,Date,Close Price,Accumulated Volume,C2 - Close Price,C2 - Accumulated Volume,C3 - Close Price,C3 - Accumulated Volume,C4 - Close Price,C4 - Accumulated Volume
64,SB,1992-01-02,8.590,5644.0,8.530,1712.0,8.480,1013.0,8.480,1013.0
65,SB,1992-01-03,8.410,5154.0,8.370,3388.0,8.370,769.0,8.370,769.0
66,SB,1992-01-06,8.500,4996.0,8.440,2751.0,8.400,762.0,8.400,762.0
67,SB,1992-01-07,8.350,4846.0,8.320,4290.0,8.340,1837.0,8.340,1837.0
68,SB,1992-01-08,8.430,5456.0,8.460,4665.0,8.460,1564.0,8.460,1564.0
...,...,...,...,...,...,...,...,...,...,...
168965,SI,2021-12-27,22.680,1.0,23.060,108793.0,23.100,111.0,23.100,111.0
168966,SI,2021-12-28,23.150,211.0,23.015,73451.0,23.100,160.0,23.100,160.0
168967,SI,2021-12-29,22.635,172.0,22.825,98976.0,22.860,45.0,22.860,45.0
168968,SI,2021-12-30,23.070,344.0,23.100,75359.0,23.075,109.0,23.075,109.0


# Create Daily Adjusted Return Metric for Backtesting Performance

Daily Adjusted Returns are calculated for the C2 contracts as those are the contracts that we trade within our strategies

In [402]:
price_data["Daily Adjusted Return"] = price_data["Close Price"] / price_data["Close Price"].shift(1) - 1

In [403]:
price_data["C2 - Daily Adjusted Return"] = price_data["C2 - Close Price"] / price_data["C2 - Close Price"].shift(1) - 1

In [404]:
price_data["C3 - Daily Adjusted Return"] = price_data["C3 - Close Price"] / price_data["C3 - Close Price"].shift(1) - 1

In [405]:
price_data["C4 - Daily Adjusted Return"] = price_data["C4 - Close Price"] / price_data["C4 - Close Price"].shift(1) - 1

In [406]:
# Create a flag that will be set to 0 whenever a return should be set to 0%
price_data["adjustment_flag"] = 1

In [407]:
price_data["matching_contract_check"] = price_data["Instrument"].shift(1)
price_data["matching_contract_check"] = price_data[[
    "matching_contract_check", "Instrument"
]].apply(
    lambda x: "OK" if x[0] == x[1] else np.nan, axis=1
)

In [408]:
price_data.dropna(subset=["Daily Adjusted Return", "matching_contract_check"], inplace=True)

In [409]:
price_data

,Instrument,Date,Close Price,Accumulated Volume,C2 - Close Price,C2 - Accumulated Volume,C3 - Close Price,C3 - Accumulated Volume,C4 - Close Price,C4 - Accumulated Volume,Daily Adjusted Return,C2 - Daily Adjusted Return,C3 - Daily Adjusted Return,C4 - Daily Adjusted Return,adjustment_flag,matching_contract_check
65,SB,1992-01-03,8.410,5154.0,8.370,3388.0,8.370,769.0,8.370,769.0,-0.020955,-0.018757,-0.012972,-0.012972,1,OK
66,SB,1992-01-06,8.500,4996.0,8.440,2751.0,8.400,762.0,8.400,762.0,0.010702,0.008363,0.003584,0.003584,1,OK
67,SB,1992-01-07,8.350,4846.0,8.320,4290.0,8.340,1837.0,8.340,1837.0,-0.017647,-0.014218,-0.007143,-0.007143,1,OK
68,SB,1992-01-08,8.430,5456.0,8.460,4665.0,8.460,1564.0,8.460,1564.0,0.009581,0.016827,0.014388,0.014388,1,OK
69,SB,1992-01-09,8.450,4213.0,8.440,4566.0,8.470,2396.0,8.470,2396.0,0.002372,-0.002364,0.001182,0.001182,1,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168965,SI,2021-12-27,22.680,1.0,23.060,108793.0,23.100,111.0,23.100,111.0,-0.006788,0.007207,0.007853,0.007853,1,OK
168966,SI,2021-12-28,23.150,211.0,23.015,73451.0,23.100,160.0,23.100,160.0,0.020723,-0.001951,0.000000,0.000000,1,OK
168967,SI,2021-12-29,22.635,172.0,22.825,98976.0,22.860,45.0,22.860,45.0,-0.022246,-0.008255,-0.010390,-0.010390,1,OK
168968,SI,2021-12-30,23.070,344.0,23.100,75359.0,23.075,109.0,23.075,109.0,0.019218,0.012048,0.009405,0.009405,1,OK


For every contract, there are rules specified as to when the contracts expire. For the adjusted daily return metric, we assume that any commodity position is exited 3 days prior to the expiration and re-entered 3 trading days post the expiration at a 0% cost. Hence, the return during this exit window will be exactly 0%

In [410]:
price_data["Year"] = price_data.Date.dt.year
price_data["Month"] = price_data.Date.dt.month
price_data["Day"] = price_data.Date.dt.day
price_data["Day of Week"] = price_data.Date.dt.day_of_week

Sugar No. 11

In [411]:
# The rules for Sugar No. 11 state that expiry is conditional on the last trading day so we need to find the last & first trading days for our exit window. This can be reused for other contracts
last_tr_days = price_data.groupby([
    "Instrument",
    "Year",
    "Month"
]).agg(
    last_tr_day = ("Day", "max"),
    first_tr_day = ("Day", "min"),
).reset_index()

In [412]:
price_data = price_data.merge(last_tr_days, how="left")

In [413]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day",
    "last_tr_day",
    "first_tr_day"
]].apply(
    lambda x: 0 if x[1] == "SB" and ((x[2] in [2, 4, 6, 9] and x[3] >= x[4] - 3) or (x[2] in [3, 5, 7, 10] and x[3] <= x[5] + 2)) else x[0],
    axis=1
)

Sugar White

In [414]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day",
    "last_tr_day",
    "first_tr_day"
]].apply(
    lambda x: 0 if x[1] == "LSU" and x[2] in [2, 4, 7, 9, 11] and x[4] - 16 + 3 >= x[3] >= x[4] - 16 - 3 else x[0],
    axis=1
)

Coffee "C"

In [415]:
# For coffee we need "8 business days prior to ..." so for that we need to rank the trading days
price_data["business_day_rank"] = price_data.groupby([
    "Instrument",
    "Year",
    "Month"
])["Day"].rank("average", ascending=True)

price_data["business_day_rev_rank"] = price_data.groupby([
    "Instrument",
    "Year",
    "Month"
])["Day"].rank("average", ascending=False)

In [416]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank"
]].apply(
    lambda x: 0 if x[1] == "KC" and x[2] in [3, 5, 7, 9, 12] and 8 + 3 >= x[3] >= 8 - 3 else x[0],
    axis=1
)

In [417]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank"
]].apply(
    lambda x: 0 if x[1] == "ICF" and x[2] in [3, 5, 7, 9, 12] and 6 + 3 >= x[3] >= 6 - 3 else x[0],
    axis=1
)

Cotton No. 2

In [418]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank"
]].apply(
    lambda x: 0 if x[1] == "CT" and x[2] in [3, 5, 7, 10, 12] and 17 + 3 >= x[3] >= 17 - 3 else x[0],
    axis=1
)

Chicago Wheat

In [419]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "W" and x[2] in [3, 5, 7, 9, 12] and 14 + 3 >= x[3] >= 14 - 3 else x[0],
    axis=1
)

Kansas Wheat

In [420]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "KW" and x[2] in [3, 5, 7, 9, 12] and 14 + 3 >= x[3] >= 14 - 3 else x[0],
    axis=1
)

Corn

In [421]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "C" and x[2] in [3, 5, 7, 9, 12] and 14 + 3 >= x[3] >= 14 - 3 else x[0],
    axis=1
)

Soybeans

In [422]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "S" and x[2] in [1, 3, 5, 7, 8, 9, 11] and 14 + 3 >= x[3] >= 14 - 3 else x[0],
    axis=1
)

Cocoa

In [423]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank"
]].apply(
    lambda x: 0 if x[1] == "CC" and x[2] in [3, 5, 7, 9, 12] and 11 + 3 >= x[3] >= 11 - 3 else x[0],
    axis=1
)

Live Cattle

In [424]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day",
    "last_tr_day",
    "first_tr_day"
]].apply(
    lambda x: 0 if x[1] == "LC" and ((x[2] in [2, 4, 6, 8, 10, 12] and x[3] >= x[4] - 3) or (x[2] in [3, 5, 7, 9, 11, 1] and x[3] <= x[5] + 2)) else x[0],
    axis=1
)

Lean Hogs

In [425]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "LH" and x[2] in [2, 4, 5, 6, 7, 8, 10, 12] and 10 + 3 >= x[3] >= 10 - 3 else x[0],
    axis=1
)

Feeder Cattle

In [426]:
# For feeder cattle we need to find the last tradeable Thursday of a contract month
# The rules for Sugar No. 11 state that expiry is conditional on the last trading day so we need to find the last & first trading days for our exit window. This can be reused for other contracts
thursdays = price_data[(price_data["Day of Week"] == 3) & (price_data["Instrument"] == "FC")].groupby([
    "Instrument",
    "Year",
    "Month"
]).agg(
    last_thursday=("Day", "max"),
).reset_index()

In [427]:
import calendar

In [428]:
thursdays = thursdays.sort_values(by=["Instrument", "Year", "Month"], ascending=True)
thursdays["previous_month_thursday"] = thursdays["last_thursday"].shift(1)
thursdays["previous_month"] = thursdays["Month"].shift(1)
thursdays["previous_year"] = thursdays["Year"].shift(1)
thursdays["days_post_thursday"] = thursdays[
    ["previous_year", "previous_month", "previous_month_thursday"]
].apply(
    lambda x: max(0, calendar.monthrange(int(x[0]), int(x[1]))[1] - x[2] - 3) if not pd.isnull(x[0]) else 0, axis=1
)
thursdays = thursdays[["Instrument", "Year", "Month", "last_thursday", "days_post_thursday"]]

In [429]:
price_data = price_data.merge(thursdays, how="left")

In [430]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day",
    "last_thursday",
    "days_post_thursday"
]].apply(
    lambda x: 0 if x[1] == "FC" and (
        (x[2] in [1, 3, 4, 5, 8, 9, 10, 11] and x[4] - 3 <= x[3] <= x[4] + 3) or
        (x[2] in [2, 4, 5, 6, 9, 10, 11, 12] and x[3] <= x[5])
    ) else x[0],
    axis=1
)

COMEX Copper

In [431]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "HG" and (
        (x[2] in [3, 5, 7, 9, 12] and 3 + 3 >= x[3] >= 3 - 3) or
        (x[2] in [4, 6, 8, 10, 1] and x[4] == 1)
    ) else x[0],
    axis=1
)

SHFE Aluminum

In [432]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "SAF" and 15 + 3 >= x[3] >= 15 - 3 else x[0],
    axis=1
)

SHFE Zinc

In [433]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "SZN" and 15 + 3 >= x[3] >= 15 - 3 else x[0],
    axis=1
)

SHFE Nickel

In [434]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "SNI" and 15 + 3 >= x[3] >= 15 - 3 else x[0],
    axis=1
)

SHFE Lead

In [435]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "SPB" and 15 + 3 >= x[3] >= 15 - 3 else x[0],
    axis=1
)

COMEX Gold

In [436]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "GC" and (
        (3 + 3 >= x[3] >= 3 - 3) or
        (x[4] == 1)
    ) else x[0],
    axis=1
)

COMEX Silver

In [437]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "SI" and (
        (3 + 3 >= x[3] >= 3 - 3) or
        (x[4] == 1)
    ) else x[0],
    axis=1
)

WTI Crude Oil

In [438]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "CL" and 22 + 3 >= x[3] >= 22 - 3 else x[0],
    axis=1
)

Natural Gas

In [439]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "NG" and (
        (3 + 3 >= x[3] >= 3 - 3) or
        (x[4] == 1)
    ) else x[0],
    axis=1
)

Brent Crude Oil

In [440]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "LCO" and (
        (3 >= x[3]) or
        (x[4] <= 3)
    ) else x[0],
    axis=1
)

Low Sulphur Gasoil

In [441]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "Day"
]].apply(
    lambda x: 0 if x[1] == "LGO" and 12 + 3 >= x[3] >= 12 - 3 else x[0],
    axis=1
)

RBOB Gasoline

In [442]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "RB" and (
        (3 >= x[3]) or
        (x[4] <= 3)
    ) else x[0],
    axis=1
)

ULS Diesel

In [443]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "HO" and (
        (3 >= x[3]) or
        (x[4] <= 3)
    ) else x[0],
    axis=1
)

Heating Oil - Low Sulphur

In [444]:
price_data["adjustment_flag"] = price_data[[
    "adjustment_flag",
    "Instrument",
    "Month",
    "business_day_rev_rank",
    "business_day_rank"
]].apply(
    lambda x: 0 if x[1] == "LHO" and (
        (4 >= x[3]) or
        (x[4] <= 2)
    ) else x[0],
    axis=1
)

# Finish Cleaned Dataset

In [445]:
# Set adjusted returns to 0 where necessary
price_data["Daily Adjusted Return"] = price_data[[
    "Daily Adjusted Return", "adjustment_flag"
]].apply(
    lambda x: x[0] if x[1] == 1 else 0,
    axis=1
)

price_data["C2 - Daily Adjusted Return"] = price_data[[
    "C2 - Daily Adjusted Return", "adjustment_flag"
]].apply(
    lambda x: x[0] if x[1] == 1 else 0,
    axis=1
)

price_data["C3 - Daily Adjusted Return"] = price_data[[
    "C3 - Daily Adjusted Return", "adjustment_flag"
]].apply(
    lambda x: x[0] if x[1] == 1 else 0,
    axis=1
)

price_data["C4 - Daily Adjusted Return"] = price_data[[
    "C4 - Daily Adjusted Return", "adjustment_flag"
]].apply(
    lambda x: x[0] if x[1] == 1 else 0,
    axis=1
)

In [446]:
price_data.columns

Index(['Instrument', 'Date', 'Close Price', 'Accumulated Volume',
       'C2 - Close Price', 'C2 - Accumulated Volume', 'C3 - Close Price',
       'C3 - Accumulated Volume', 'C4 - Close Price',
       'C4 - Accumulated Volume', 'Daily Adjusted Return',
       'C2 - Daily Adjusted Return', 'C3 - Daily Adjusted Return',
       'C4 - Daily Adjusted Return', 'adjustment_flag',
       'matching_contract_check', 'Year', 'Month', 'Day', 'Day of Week',
       'last_tr_day', 'first_tr_day', 'business_day_rank',
       'business_day_rev_rank', 'last_thursday', 'days_post_thursday'],
      dtype='object')

In [447]:
price_data[['Instrument', 'Date', 'Year', 'Month',
            'Day', 'Day of Week', 'Close Price', 'Accumulated Volume',
            'C2 - Close Price', 'C2 - Accumulated Volume', 'C3 - Close Price',
            'C3 - Accumulated Volume', 'C4 - Close Price',
            'C4 - Accumulated Volume', 'Daily Adjusted Return',
            'C2 - Daily Adjusted Return', 'C3 - Daily Adjusted Return',
            'C4 - Daily Adjusted Return']].to_csv(
    "price_data_cleaned.csv", index=False
)